# Data Reading Process

Importing libraries

In [1]:
import dask
import dask.dataframe as dd
import os
from dask.distributed import Client, LocalCluster
from dask_ml.impute import SimpleImputer
from dask_ml.compose import make_column_transformer

Declaring local cluster instance to manipulate and process parallel data

In [2]:
cluster = LocalCluster(n_workers = 8, threads_per_worker = 2, memory_limit = '8GB')
client = Client(cluster)

Show Dashboard link

In [3]:
client.dashboard_link

'http://127.0.0.1:8787/status'

creating a function to restart client

Importing databases to use to create de machine learning model

In [3]:
# reading all tables from database
table_names = os.listdir('databases')[1:]

# importing all databases
tables = {tbl_name.split('.')[0]: client.persist(dd.read_csv(f"databases/{tbl_name}", blocksize ='100MB')) for tbl_name in table_names}

# Data Wrangling

Joining tables to create a single one to use as the main dataframe

In [4]:
# merging all tables
data = tables['balances'].merge(tables['liabilities'], on = ['ID', 'period', 'month'], how = 'left')\
                        .merge(tables['movements'], on = ['ID', 'period', 'month'], how = 'left')\
                        .merge(tables['customers'], on = ['ID'], how = 'left')\
                        .merge(tables['digital'], on = ['ID', 'period', 'month'], how = 'left').repartition(25)

# removing from memory singular tables by deleting
del tables

Showing five records

In [ ]:
data.head()

2023-10-08 02:23:54,801 - tornado.application - ERROR - Exception in callback <bound method BokehTornado._keep_alive of <bokeh.server.tornado.BokehTornado object at 0x00000143577C2B90>>
Traceback (most recent call last):
  File "C:\Users\Acer\anaconda3\envs\machine_learning\Lib\site-packages\tornado\ioloop.py", line 919, in _run
    val = self.callback()
          ^^^^^^^^^^^^^^^
  File "C:\Users\Acer\anaconda3\envs\machine_learning\Lib\site-packages\bokeh\server\tornado.py", line 779, in _keep_alive
    c.send_ping()
  File "C:\Users\Acer\anaconda3\envs\machine_learning\Lib\site-packages\bokeh\server\connection.py", line 91, in send_ping
    self._socket.ping(str(self._ping_count).encode("utf-8"))
  File "C:\Users\Acer\anaconda3\envs\machine_learning\Lib\site-packages\tornado\websocket.py", line 439, in ping
    raise WebSocketClosedError()
tornado.websocket.WebSocketClosedError
2023-10-08 02:23:55,833 - tornado.application - ERROR - Exception in callback <bound method BokehTornado._k

## Null Values Imputation

Checking data types and null values

In [7]:
dd.concat([data.dtypes, data.isnull().sum()/ len(data)], axis = 1).rename(columns = {0: 'data_type', 1: 'null_ratio'}).compute()

,data_type,null_ratio
ID,string,0.000000
age,float64,0.000000
balance_amount,float64,0.000000
bureau_risk,string,0.000000
days_default,float64,0.000000
dig_1,float64,0.247692
dig_10,float64,0.247692
dig_11,float64,0.247692
dig_2,float64,0.247692
dig_3,float64,0.247692


Before deciding what kind of impution it is going to be performed, we observe its quartils to decide if imputing with mean or median

In [12]:
# selecting group of columns based on their name
dig_columns = [column for column in data.columns if 'dig' in column]
prod_columns = ['product_1_x', 'product_2_x']
type_columns = ['type_1', 'type_2', 'type_3', 'type_4']

# describing selected columns
data[dig_columns + prod_columns + type_columns].describe().compute()

,dig_1,dig_2,dig_3,dig_4,dig_5,dig_6,dig_7,dig_8,dig_9,dig_10,dig_11,product_1_x,product_2_x,type_1,type_2,type_3,type_4
count,8.725112e+06,8.725112e+06,8.725112e+06,8.725112e+06,8.725112e+06,8.725112e+06,8.725112e+06,8.725112e+06,8.725112e+06,8.725112e+06,8.725112e+06,1.041170e+07,1.041170e+07,5.199137e+06,5.199137e+06,5.199137e+06,5.199137e+06
mean,1.729927e+00,3.250468e+00,3.033453e+00,2.620164e+00,1.598258e+00,3.604106e+00,2.253315e+00,2.012096e+00,3.075644e+00,1.535400e+00,2.711378e+00,3.038695e+00,5.321534e-01,2.075979e+00,2.599862e+00,1.599407e+00,1.741834e+00
std,1.181385e+00,8.854857e-01,8.896423e-01,1.073410e+00,1.103790e+00,1.116196e+00,1.490340e+00,1.398692e+00,1.057191e+00,1.587534e+00,1.256354e+00,1.209978e+00,1.316910e+00,1.884979e+00,1.751632e+00,1.857044e+00,1.931270e+00
min,0.000000e+00,0.000000e+00,-1.000000e-04,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e-04,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,9.050000e-01,2.813500e+00,2.526800e+00,1.949600e+00,8.656000e-01,3.226900e+00,1.029200e+00,1.063700e+00,2.431100e+00,0.000000e+00,1.941900e+00,2.463400e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,1.555900e+00,3.396900e+00,3.107200e+00,2.519800e+00,1.391400e+00,3.869900e+00,2.389800e+00,2.058500e+00,3.187100e+00,1.313200e+00,2.833700e+00,3.217550e+00,0.000000e+00,2.576200e+00,3.192100e+00,0.000000e+00,0.000000e+00
75%,2.385800e+00,3.893900e+00,3.666400e+00,3.238200e+00,2.155725e+00,4.335250e+00,3.429000e+00,3.061500e+00,3.878800e+00,2.789400e+00,3.660300e+00,3.904750e+00,0.000000e+00,3.806450e+00,3.996800e+00,3.451400e+00,3.711700e+00
max,5.000000e+00,5.000000e+00,5.000000e+00,5.000000e+00,5.000000e+00,5.000000e+00,5.000000e+00,5.000000e+00,5.000000e+00,5.000000e+00,5.000000e+00,5.000000e+00,5.000000e+00,5.000000e+00,5.000000e+00,5.000000e+00,5.000000e+00


As it can be seen, data has no skew apparently beacuse of the proximity between max value and quartil 3, and mean an median are quite the same, so median is chosen to be de imputer

In [ ]:
# joining all selected columns
missing_numeric_columns = dig_columns + prod_columns + type_columns
# declaring column trasnformer
column_transformer = make_column_transformer((SimpleImputer(strategy = 'mean'), missing_numeric_columns), 
                                            remainder = 'passthrough').set_output(transform = 'pandas')

# imputing values
data = column_transformer.fit_transform(data)